<a href="https://colab.research.google.com/github/anelglvz/Working-Analyst/blob/main/ML-AI-for-the-Working-Analyst/Semana6_1_Working_Analyst_ComparacionesUsuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Introducción

En este ejemplo utilizaremos los datos de la [competencia de Netflix](https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data?select=combined_data_1.txt) en Kaggle. El objetivo de esta competencia era mejorar el algoritmo de recomendación 10%. Nosotros no seremos tan avariciosos. En esta primer sesión exploraremos comparaciones básicas que podríamos utilizar. Para la segunda sesión los objetivos serán: 

*   Análisis exploratorio de matriz y por qué es dispersa.
*   Implementación de Singular Value Decomposition.
*   Implementación de un modelo de sistema de recomendación de filtro colaborativo.
*   Generar la predicción de recomendaciones con buen resultado y no.

In [ ]:
import math 
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


En este caso cargar los datos nos llevará un rato por la gran cantidad de registros que tenemos. Sólo cargaremos un archivo, pero la competencia tiene un total de 3 archivos.

In [ ]:
# Recuerde, pueden conseguir la dirección del archivo en su Drive y copiarla en la dirección
df = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana5/combined_data_1.txt', 
                 names=['Client_Id', 'Rating', 'Date'], low_memory=True, nrows=12*(10**6))

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# Revisamos los nulos. 

df.isnull().sum()

En este caso los registros que tenemos con valores nulos corresponden al Id de las películas. Si observamos los registros nulos están ordenados del 1 al 2,340. 

In [ ]:
df[df['Rating'].isnull()].shape

In [ ]:
df[df['Rating'].isnull()]

In [ ]:
movies = df['Rating'].isnull().sum()
print(f'Este es el número de películas que tenemos en este archivo: {movies}')

In [ ]:
reviews = df[df['Rating'].isnull()==False]['Client_Id'].count()
print(f'Este es el número de calificaciones: {reviews}')

In [ ]:
users = df['Client_Id'].nunique() - movies
print(f'Esta es la cantidad de usuarios que tenemos: {users}')

En este caso no estaremos trabajando con las fechas. Por lo tanto haremos un subconjunto de nuestro DF original. 

In [ ]:
df_sub = df[['Client_Id', 'Rating']]

In [ ]:
df_sub

Ahora observemos como se distribuye la frecuencia para la columna de rating.

In [ ]:
sns.countplot(y=df_sub['Rating'], orient='v', palette='Blues');

In [ ]:
df_sub['Rating'].value_counts() / df_sub['Rating'].count() * 100

La mayor parte de nuestros valores se distribuyen en los ratings de 3 y 4 estrellas. La mayoría de los rating son positivos.

### Hora de la limpieza

En este caso los valores nulos que tenemos hacen referencia a **Id** de 'clientes' con rating vacíos. En realidad esta información es el **Id** de la película. Lo que debemos hacer ahora es quitar estos registros y añadir los **Id's** de película como una nueva columna. 

In [ ]:
# Generamos una serie con valores booleanos. Donde Verdadero será igual al lugar
# donde hay un Id de película.
pd.isnull(df_sub['Rating'][:100])

In [ ]:
# Colocamos la serie como un DataFrame
df_null = pd.DataFrame(pd.isnull(df_sub['Rating']))
df_null.head()

In [ ]:
# Obtenemos sólo los registros de películas junto con índice hasta donde llega 
# los rating para esa película.
df_null = df_null[df_null['Rating'] == True]
df_null.head()

In [ ]:
# Colocamos el índice como columna para tener la ubicación para hasta donde repe-
# tir nuestros valores de Id para esa película.
df_null = df_null.reset_index()
df_null.head()

In [ ]:
df_null.tail()

In [ ]:
movie_id_array = [] # Generamos una lista vacía donde colocaremos el Id de la película las veces que se repita.
movie_id = 1 # Inicializamos un contador

# En esta celda para saber cómo hace el proceso imprimo los distintos pasos.
for i, j in zip(df_null['index'][1:], df_null['index'][:-1]): # Iteramos sobre los valores de la columna 'index'
                                                               # empezando por el valor n+1 y en segundo lugar desde n hasta el penúltimo valor de la serie.
  temporary = np.full((1, i-j-1), fill_value=movie_id) # Creamos una matriz llena de valores con la forma de 1x(la diferencia del valor (n+1)-n-1).                                                     # Esto nos da una matriz llena con el Id repetido el número de reviews para esa película.
  movie_id_array = np.append(movie_id_array, temporary) # Lo añadimos a una lista. 
  movie_id += 1                                         # Aumentamos el Id para la siguiente película.

In [ ]:
movie_id

In [ ]:
# Generamos los Id's para la última película que no está contemplada en nuestro loop.
last_movie = np.full((1, len(df_sub) - df_null.iloc[-1, 0] - 1), fill_value=movie_id)
movie_id_array = np.append(movie_id_array, last_movie)

In [ ]:
movie_id_array.shape # la cantidad de Id's corresponde con la cantidad de películas.

In [ ]:
movie_id_array

Ahora para tener todo en orden es necesario que quitemos los registros nulos de nuestro df y añadamos los Id's que generamos.

In [ ]:
df_clean = df_sub[pd.notnull(df_sub['Rating'])].copy()

In [ ]:
df_clean

In [ ]:
df_clean['Movie_Id'] = movie_id_array.astype('int16')

In [ ]:
df_clean['Client_Id'] = df_clean['Client_Id'].astype('int32')

In [ ]:
df_clean.head(600)

In [ ]:
df_clean.tail()

In [ ]:
df_clean.info()

### Convirtiendo a matriz dispersa o generando nuestra tabla Usuario-Item

In [ ]:
df_clean

In [ ]:
df_short = df_clean[df_clean['Movie_Id']<1000]

In [ ]:
df_short

In [ ]:
# Por limitantes de la ramm, no podemos crearlo, pero en local o usando otras herramientas, podrían 
%%time
pivot_user_item = df_short.pivot(index='Client_Id', columns='Movie_Id', values='Rating').fillna(0)

In [ ]:
pivot_user_item

In [ ]:
pivot_copy = pivot_user_item.copy()

In [ ]:
for i in range(1,9):
  pivot_copy = pivot_copy[pivot_copy[i] != 0]

In [ ]:
pivot_copy

## Pequeñas Pruebas

In [ ]:
pivot_copy = pivot_copy[[1,2,3,4,5,6,7,8,9]]
pivot_copy

In [ ]:
plt.plot(range(1,10), pivot_copy.loc[305344,:])
plt.plot(range(1,10), pivot_copy.loc[387418,:])
plt.plot(range(1,10), pivot_copy.loc[1664010,:])
plt.plot(range(1,10), pivot_copy.loc[2118461,:])
plt.plot(range(1,10), pivot_copy.loc[2439493,:])

plt.legend(['Client 305344', 'Client 387418', 'Client 1664010', 'Client 2118461', 'Client 2439493'], bbox_to_anchor=(1.04,1), loc="upper left")

## Comparaciones con medida Coseno

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

Se obtiene de la siguiente manera:

$$ S_C (P,Q):= \cos(\theta) = {P \cdot Q \over \|P\| \|Q\|} = \frac{ \sum\limits_{i=1}^{n}{p_i  q_i} }{ \sqrt{\sum\limits_{i=1}^{n}{p_i^2}}  \sqrt{\sum\limits_{i=1}^{n}{q_i^2}} }$$

Donde $\theta$ es el angulo entre los vectores $P$ y $Q$.

In [ ]:
help(cosine_similarity)

Ejemplificación:

In [ ]:
# Comparemos los vectores (1,1) y (-1,1)
V = np.array([[1,1], [-1,1]])
v1_pos = [0, 0]
v2_pos = [0, 0]

plt.plot(0,0,'ok')
plt.quiver(v1_pos, v2_pos, V[:,0], V[:,1], color=['r','b'], scale=5)
plt.grid()
plt.show()

In [ ]:
# La restaremos al 1 por la forma en que lo calcula el módulo que estamos utilizando (scipy)
cosine_similarity([np.array([1,1]), np.array([-1,1])])

In [ ]:
# Comparemos los vectores (1,1) y (1,2)
V = np.array([[1,1], [1,2]])
#V = np.array([[5,5], [1,2]])
v1_pos = [0,0]
v2_pos = [0,0]

plt.plot(0,0,'ok')
plt.quiver(v1_pos, v2_pos, V[:,0], V[:,1], color=['r','b'], scale=8)
plt.grid()
plt.show()

In [ ]:
cosine_similarity([np.array([1,1]), np.array([1,2])])

In [ ]:
# Comparemos los vectores (1,1), (1, -0.5)
V = np.array([[1,1], [-1,-0.5]])
v1_pos = [0,0]
v2_pos = [0,0]

plt.plot(0,0,'ok')
plt.quiver(v1_pos, v2_pos, V[:,0], V[:,1], color=['r','b'], scale=8)
plt.grid()
plt.show()

In [ ]:
cosine_similarity([np.array([1,1]), np.array([-1,-.5])])

# Usemos la similitud coseno con nuestros datos:

In [ ]:
# Recordemos los datos que nos quedaron tras un filtro muy estricto
pivot_copy

In [ ]:
de_1_a_2 = cosine_similarity([pivot_copy.loc[305344,:], pivot_copy.loc[387418,:]])
de_1_a_2

In [ ]:
de_1_a_4 = cosine_similarity([pivot_copy.loc[305344,:], pivot_copy.loc[2118461,:]])
de_1_a_4

In [ ]:
de_3_a_4 = cosine_similarity([pivot_copy.loc[1664010,:], pivot_copy.loc[2118461,:]])
de_3_a_4

In [ ]:
M = np.zeros([5,5])

for i in range(5):
  for j in range(5):
    M[i,j] = cosine_similarity([pivot_copy.iloc[i,:], pivot_copy.iloc[j,:]])[1][0]

print(M)

In [ ]:
pivot_copy = pivot_copy - 2.5
pivot_copy

In [ ]:
M = np.zeros([5,5])

for i in range(5):
  for j in range(5):
    M[i,j] = cosine_similarity([pivot_copy.iloc[i,:], pivot_copy.iloc[j,:]])[1][0]

print(M)